# Equilibrate extension
Notebook to illustrate the calculation of forcing a silicate liquid to be in equilibrium with both quartz and corundum at a specifed temperature and pressure

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
%matplotlib notebook

### Get class instances to calculate the properties of quartz and corundum from Berman (1988)

In [ ]:
from thermoengine import phases

In [ ]:
thermoDB = phases.ThermoDB()
quartz = thermoDB.new_phase('Qz')
corundum = thermoDB.new_phase('Crn')

### Get class instances to calculate the properties of silicate liquid from MELTS 1.0.2

In [ ]:
import ctypes
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

In [ ]:
LiquidMelts = ObjCClass('LiquidMelts')
liquid = LiquidMelts.alloc().init()

### Set number of components in the system and the number of fixed phases

In [ ]:
c = 5
f = 2

### The amount of quartz and corundum in the system is arbitrary ...
so, we fix the amount of each phase at a constant value

In [ ]:
nPhaseFix = 1.0

### Choose and initial bulk composition for the silicate liquid 
The initial composition listed below is a high-silica rhyolite projected into the system SiO<sub>2</sub>-Al<sub>2</sub>O<sub>3</sub>-CaO-Na<sub>2</sub>O-K<sub>2</sub>O

In [ ]:
nc = liquid.numberOfSolutionComponents()
bc = np.zeros((c,1))
def setBulkComposition(nSiO2=0.665792, nAl2O3=0.042436, nQz=nPhaseFix, nCr=nPhaseFix):
    bc[0] = nSiO2 + 0.004596 + 0.038493 + 0.062105 + nQz # SiO2 = SiO2 + CaSiO3 + Na2SiO3 + KAlSiO4
    bc[1] = nAl2O3 + 0.062105/2.0 + nCr                  # Al2O3 = Al2O3 + KAlSiO4/2
    bc[2] = 0.004596                                     # CaO = CaSiO3
    bc[3] = 0.038493                                     # Na2O = Na2SiO3
    bc[4] = 0.062105/2.0                                 # K2O = KAlSiO4/2
m = (ctypes.c_double*nc)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
def setMoles(nSiO2=0.665792, nAl2O3=0.042436):
    nLiq = np.zeros((c,1))
    nLiq[0] = nSiO2
    nLiq[1] = nAl2O3
    nLiq[2] = 0.004596
    nLiq[3] = 0.038493
    nLiq[4] = 0.062105
    m[0] = nLiq[0]  # SiO2
    m[1] = 0.0      # TiO2
    m[2] = nLiq[1]  # Al2O3
    m[3] = 0.0      # Fe2O3
    m[4] = 0.0      # Cr2O3
    m[5] = 0.0      # Fe2SiO4
    m[6] = 0.0      # Mn2SiO4
    m[7] = 0.0      # Mg2SiO4
    m[8] = 0.0      # NiSi1/2O2
    m[9] = 0.0      # CoSi1/2O2
    m[10] = nLiq[2] # CaSiO3
    m[11] = nLiq[3] # Na2SiO3
    m[12] = nLiq[4] # KAlSiO4
    m[13] = 0.0     # Ca3(PO4)2
    m[14] = 0.0     # H2O
    return nLiq

### Choose a temperature and pressure
The thermodynamic properties of quartz are only functions of temperature and pressure

In [ ]:
t = 1100.0 # K
p = 1750.0 # bars
mu0Qz = quartz.get_gibbs_energy(t, p)
mu0Cr = corundum.get_gibbs_energy(t, p)

### Find an initial guess that is consistent with the constraints

In [ ]:
def fcon(x):
    setMoles(nSiO2=x[0], nAl2O3=x[1])
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    return (mu0Qz-muLiq.valueAtIndex_(0))**2 + (mu0Cr-muLiq.valueAtIndex_(2))**2
result = opt.minimize(fcon,np.array([0.665792, 0.042436]))
print (result)
nLiqSiO2 = result.x[0]
nLiqAl2O3 = result.x[1]
nQz = nPhaseFix
nCr = nPhaseFix
setBulkComposition(nSiO2=nLiqSiO2, nAl2O3=nLiqAl2O3, nQz=nQz, nCr=nCr)
print (bc)

### Set up constraint matrix - matrix is constant
Columns: nLiqSiO<sub>2</sub>, nLiqAl<sub>2</sub>O<sub>3</sub>, nLiqCaSiO<sub>3</sub>, nLiqNa<sub>2</sub>SiO<sub>3</sub>, nLiqKAlSiO<sub>4</sub>, nQz, nCr  
Rows: SiO<sub>2</sub>, Al<sub>2</sub>O<sub>3</sub>, CaO, Na<sub>2</sub>O, K<sub>2</sub>O
$$
\left[ {\begin{array}{*{20}{c}}
{{b_{{\rm{Si}}{{\rm{O}}_2}}}}\\
{{b_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}}}\\
{{b_{{\rm{CaO}}}}}\\
{{b_{{\rm{N}}{{\rm{a}}_2}{\rm{O}}}}}\\
{{b_{{{\rm{K}}_2}{\rm{O}}}}}
\end{array}} \right] = \left[ {\begin{array}{*{20}{c}}
1&0\\
0&1\\
0&0\\
0&0\\
0&0
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{{n_{Qz}}}\\
{{n_{Cr}}}
\end{array}} \right] + \left[ {\begin{array}{*{20}{c}}
1&0&1&1&1\\
0&1&0&0&{\frac{1}{2}}\\
0&0&1&0&0\\
0&0&0&1&0\\
0&0&0&0&{\frac{1}{2}}
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}\\
{n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}
\end{array}} \right]
$$

In [ ]:
C = np.array([[1,0,1,1,1,1,0],[0,1,0,0,0.5,0,1],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0.5,0,0]])

### ...  and project it into the null space of fixed constraints - yields a constant matrix

In [ ]:
CTf = C[:,c:].transpose()
U,S,VT = np.linalg.svd(CTf)
rank = np.count_nonzero(S)
VTff = VT[rank:,:]
Cproj = np.matmul(VTff, C)
print (Cproj)
bcproj = np.matmul(VTff, bc)
print (bcproj)

### Define the Khorzhinski potential function and its gradient
$$
L = {G^{liq}} + {n_{Qz}}\left( {\mu _{Qz}^o - \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}} \right) + {n_{Cr}}\left( {\mu _{Cr}^o - \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}} \right)
$$

In [ ]:
def Khorzhinskii(nQz=nPhaseFix, nCr=nPhaseFix, nLiqSiO2=0.665792, nLiqAl2O3=0.042436, t=1100.0, p=1750.0):
    setMoles(nLiqSiO2,nLiqAl2O3) # fills m array
    Gliq = liquid.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p)
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    muLiqSiO2 = muLiq.valueAtIndex_(0)
    muLiqAl2O3 = muLiq.valueAtIndex_(2)
    result = Gliq + nQz*(mu0Qz-muLiqSiO2) + nCr*(mu0Cr-muLiqAl2O3)
    return result

### Define the gradient of the Khorzhinskii potential
$$
{\bf{g}} = \left[ {\begin{array}{*{20}{c}}
{\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}} - {n_{Cr}}\frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}\\
{\mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}} - {n_{Cr}}\frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}\\
{\mu _{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}\\
{\mu _{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}\\
{\mu _{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}\\
{\mu _{Qz}^o - \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}\\
{\mu _{Cr}^o - \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}
\end{array}} \right]
$$

In [ ]:
def Gradient(nQz=nPhaseFix, nCr=nPhaseFix, nLiqSiO2=0.665792, nLiqAl2O3=0.042436, t=1100.0, p=1750.0):
    setMoles(nLiqSiO2,nLiqAl2O3) # fills m array
    dgdm = liquid.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    result = np.zeros((c+f,1))
    result[0] = dgdm.valueAtIndex_(0) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(0, 0) \
                                      - nCr*d2gdm2.valueAtRowIndex_andColIndex_(0, 2)
    result[1] = dgdm.valueAtIndex_(2) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(2, 0) \
                                      - nCr*d2gdm2.valueAtRowIndex_andColIndex_(2, 2)
    result[2] = dgdm.valueAtIndex_(10)
    result[3] = dgdm.valueAtIndex_(11)
    result[4] = dgdm.valueAtIndex_(12)
    result[5] = mu0Qz - dgdm.valueAtIndex_(0)
    result[6] = mu0Cr - dgdm.valueAtIndex_(2)
    return result

### Define a function to compute the "A" constraint matrix
$$
{\bf{A}} = \left[ {\begin{array}{*{20}{c}}
0&0&0&0&{\frac{1}{2}}&0&0\\
0&0&0&1&0&0&0\\
0&0&1&0&0&0&0\\
{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&0\\
{ - \frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&0\\
0&0&0&0&0&1&0\\
0&0&0&0&0&0&1
\end{array}} \right]
$$

In [ ]:
def Amatrix(nLiqSiO2=0.665792, nLiqAl2O3=0.042436, t=1100.0, p=1750.0):
    setMoles(nLiqSiO2,nLiqAl2O3) # fills m array
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    bottom = np.zeros((2*f,c+f))
    bottom[0][0] = d2gdm2.valueAtRowIndex_andColIndex_(0, 0)
    bottom[0][1] = d2gdm2.valueAtRowIndex_andColIndex_(0, 2)
    bottom[0][2] = d2gdm2.valueAtRowIndex_andColIndex_(0, 10)
    bottom[0][3] = d2gdm2.valueAtRowIndex_andColIndex_(0, 11)
    bottom[0][4] = d2gdm2.valueAtRowIndex_andColIndex_(0, 12)
    bottom[1][0] = d2gdm2.valueAtRowIndex_andColIndex_(2, 0)
    bottom[1][1] = d2gdm2.valueAtRowIndex_andColIndex_(2, 2)
    bottom[1][2] = d2gdm2.valueAtRowIndex_andColIndex_(2, 10)
    bottom[1][3] = d2gdm2.valueAtRowIndex_andColIndex_(2, 11)
    bottom[1][4] = d2gdm2.valueAtRowIndex_andColIndex_(2, 12)
    bottom[2][5] = 1.0
    bottom[3][6] = 1.0
    result = np.vstack((Cproj, bottom))
    return result

### Define the Lagrangian of the Khorzhimskii function
$$
\Lambda  = {G^{liq}} - \lambda _{1:f}^T\left( {{\bf{V}}_{\left. f \right|f}^T{\bf{Cn}} - {\bf{V}}_{\left. f \right|f}^T{\bf{b}}} \right) + \left( {{n_{Qz}} - {\lambda _{{\rm{Qz}}}}} \right)\left( {\mu _{Qz}^o - \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}} \right) + \left( {{n_{Cr}} - {\lambda _{{\rm{Cr}}}}} \right)\left( {\mu _{Cr}^o - \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}} \right) - {\lambda _{{n_{{\rm{Qz}}}}}}\left( {{n_{Qz}} - 1} \right) - {\lambda _{{n_{{\rm{Cr}}}}}}\left( {{n_{Cr}} - 1} \right)
$$

In [ ]:
def Lagrangian(nQz=nPhaseFix, nCr=nPhaseFix, nLiqSiO2=0.665792, nLiqAl2O3=0.042436, t=1100.0, p=1750.0):
    nFix = np.empty((f,1))
    nFix[0] = nQz
    nFix[1] = nCr
    nLiq = setMoles(nLiqSiO2,nLiqAl2O3)
    n = np.vstack((nLiq,nFix))
    Gliq = liquid.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p)
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    muLiqSiO2 = muLiq.valueAtIndex_(0)
    muLiqAl2O3 = muLiq.valueAtIndex_(2)
    result = Gliq + nQz*(mu0Qz-muLiqSiO2) + nCr*(mu0Cr-muLiqAl2O3)
    # now add the Lagrange terms
    temp = np.matmul(Cproj, n)
    temp = np.subtract(temp, bcproj)
    result -= (np.dot(temp.transpose(), xLambda[0:3])[0][0])
    result -= xLambda[3][0]*(mu0Qz-muLiqSiO2) 
    result -= xLambda[4][0]*(mu0Cr-muLiqAl2O3)
    result -= xLambda[5][0]*(nQz-1.0)
    result -= xLambda[6][0]*(nCr-1.0)
    return result

### Define the Wronskian of the Lagrangian function
Elements of the Wronskian are given by
$$
\frac{{{\partial ^2}\Lambda }}{{\partial {n_i}\partial {n_j}}} = \frac{{{\partial ^2}{G^{liq}}}}{{\partial {n_i}\partial {n_j}}} - \frac{{\partial {n_{Qz}}}}{{\partial {n_i}}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial {n_j}}} - \frac{{\partial {n_{Qz}}}}{{\partial {n_j}}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial {n_i}}} - \left( {{n_{Qz}} - {\lambda _{{\rm{Qz}}}}} \right)\frac{{{\partial ^2}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial {n_i}\partial {n_j}}} - \frac{{\partial {n_{Cr}}}}{{\partial {n_i}}}\frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial {n_j}}} - \frac{{\partial {n_{Cr}}}}{{\partial {n_j}}}\frac{{\partial \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial {n_i}}} - \left( {{n_{Cr}} - {\lambda _{{\rm{Cr}}}}} \right)\frac{{{\partial ^2}\mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}{{\partial {n_i}\partial {n_j}}}
$$

In [ ]:
def Wronskian(nQz=nPhaseFix, nCr=nPhaseFix, nLiqSiO2=0.665792, nLiqAl2O3=0.042436, t=1100.0, p=1750.0):
    w = np.zeros((c+f,c+f))
    
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    w[0][0] =  d2gdm2.valueAtRowIndex_andColIndex_( 0,  0) # liq SiO2,    SiO2
    w[0][1] =  d2gdm2.valueAtRowIndex_andColIndex_( 0,  2) # liq SiO2,    Al2O3
    w[0][2] =  d2gdm2.valueAtRowIndex_andColIndex_( 0, 10) # liq SiO2,    CaSiO3
    w[0][3] =  d2gdm2.valueAtRowIndex_andColIndex_( 0, 11) # liq SiO2,    Na2SiO3
    w[0][4] =  d2gdm2.valueAtRowIndex_andColIndex_( 0, 12) # liq SiO2,    KAlSiO4
    w[0][5] = -d2gdm2.valueAtRowIndex_andColIndex_( 0,  0) # liq SiO2,    quartz
    w[0][6] = -d2gdm2.valueAtRowIndex_andColIndex_( 2,  0) # liq SiO2,    corundum
    w[1][1] =  d2gdm2.valueAtRowIndex_andColIndex_( 2,  2) # liq Al2O3,   Al2O3
    w[1][2] =  d2gdm2.valueAtRowIndex_andColIndex_( 2, 10) # liq Al2O3,   CaSiO3
    w[1][3] =  d2gdm2.valueAtRowIndex_andColIndex_( 2, 11) # liq Al2O3,   Na2SiO3
    w[1][4] =  d2gdm2.valueAtRowIndex_andColIndex_( 2, 12) # liq Al2O3,   KAlSiO4
    w[1][5] = -d2gdm2.valueAtRowIndex_andColIndex_( 0,  2) # liq Al2O3,   quartz
    w[1][6] = -d2gdm2.valueAtRowIndex_andColIndex_( 2,  2) # liq Al2O3,   corundum
    w[2][2] =  d2gdm2.valueAtRowIndex_andColIndex_(10, 10) # liq CaSiO3,  CaSiO3
    w[2][3] =  d2gdm2.valueAtRowIndex_andColIndex_(10, 11) # liq CaSiO3,  Na2SiO3
    w[2][4] =  d2gdm2.valueAtRowIndex_andColIndex_(10, 12) # liq CaSiO3,  KAlSiO4
    w[2][5] = -d2gdm2.valueAtRowIndex_andColIndex_( 0, 10) # liq CaSiO3,  quartz
    w[2][6] = -d2gdm2.valueAtRowIndex_andColIndex_( 2, 10) # liq CaSiO3,  corundum
    w[3][3] =  d2gdm2.valueAtRowIndex_andColIndex_(11, 11) # liq Na2SiO3, Na2SiO3
    w[3][4] =  d2gdm2.valueAtRowIndex_andColIndex_(11, 12) # liq Na2SiO3, KAlSiO4
    w[3][5] = -d2gdm2.valueAtRowIndex_andColIndex_( 0, 11) # liq Na2SiO3, quartz
    w[3][6] = -d2gdm2.valueAtRowIndex_andColIndex_( 2, 11) # liq Na2SiO3, corundum
    w[4][4] =  d2gdm2.valueAtRowIndex_andColIndex_(12, 12) # liq KAlSiO4, KAlSiO4
    w[4][5] = -d2gdm2.valueAtRowIndex_andColIndex_( 0, 12) # liq KAlSiO4, quartz
    w[4][6] = -d2gdm2.valueAtRowIndex_andColIndex_( 2, 12) # liq KAlSiO4, corundum
    
    d3gdm3 = liquid.getD3gDm3FromMolesOfComponents_andT_andP_(m, t, p)
    w[0][0] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 0, 0)
    w[0][1] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 0, 2)
    w[0][2] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 0,10)
    w[0][3] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 0,11)
    w[0][4] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 0,12)
    w[1][1] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 2, 2)
    w[1][2] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 2,10)
    w[1][3] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 2,11)
    w[1][4] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, 2,12)
    w[2][2] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0,10,10)
    w[2][3] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0,10,11)
    w[2][4] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0,10,12)
    w[3][3] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0,11,11)
    w[3][4] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0,11,12)
    w[4][4] -= (nQz-xLambda[3][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0,12,12)
    
    w[0][0] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 0, 0)
    w[0][1] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 0, 2)
    w[0][2] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 0,10)
    w[0][3] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 0,11)
    w[0][4] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 0,12)
    w[1][1] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 2, 2)
    w[1][2] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 2,10)
    w[1][3] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 2,11)
    w[1][4] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, 2,12)
    w[2][2] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2,10,10)
    w[2][3] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2,10,11)
    w[2][4] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2,10,12)
    w[3][3] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2,11,11)
    w[3][4] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2,11,12)
    w[4][4] -= (nCr-xLambda[4][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2,12,12)
    
    for i in range(0,c+f):
        for j in range(i+1,c+f):
             w[j][i] = w[i][j]
    
    return w

### Form the QR decomposition of A
Note that A must be square, so zero filled rows are added 
$
{\bf{A}} = {\bf{QR}} = \left[ {\begin{array}{*{20}{c}}
{{{\bf{Q}}_1}}&{{{\bf{Q}}_2}}
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{{{\bf{R}}_1}}\\
{\bf{0}}
\end{array}} \right]
$
and 
${{\bf{Q}}_2}^T$ is the matrix ${\bf{Z}}$

In [ ]:
A = Amatrix(nLiqSiO2=nLiqSiO2, nLiqAl2O3=nLiqAl2O3)
#bottom = np.zeros((f,c+f))
#A = np.vstack((A, bottom))
#Q,R = lin.qr(A,mode='full')
R,Q = lin.rq(A,mode='full')
print ("Q matrix:")
for i in range(0,Q.shape[0]):
    if Q.shape[1] == c+f:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e} {6:10.3e}".format( \
            Q[i][0], Q[i][1], Q[i][2], Q[i][3], Q[i][4], Q[i][5], Q[i][6]))
    elif Q.shape[1] == c:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e}".format( \
            Q[i][0], Q[i][1], Q[i][2], Q[i][3], Q[i][4]))
print ("R matrix:")
for i in range(0,R.shape[0]):
    if R.shape[1] == c+f:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e} {6:10.3e}".format( \
            R[i][0], R[i][1], R[i][2], R[i][3], R[i][4], R[i][5], R[i][6]))
    elif R.shape[1] == c:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e}".format( \
            R[i][0], R[i][1], R[i][2], R[i][3], R[i][4]))   
print ("Z matrix")
Z = Q[0:0,:].transpose()
for i in range(0,Z.shape[0]):
    if Z.shape[1] == f:
        print ("{0:10.3e} {1:10.3e}".format(Z[i][0], Z[i][1]))

### Form the vector of Lagrange multipliers
$$
{\bf{g}} = {{\bf{A}}^T}\left[ {\begin{array}{*{20}{c}}
{{\lambda _{{n_1}}}}\\
 \vdots \\
{{\lambda _{{n_{c - f}}}}}\\
{{\lambda _{{\phi _1}}}}\\
 \vdots \\
{{\lambda _{{\phi _f}}}}
\end{array}} \right]
$$

In [ ]:
print ("Gradient:")
g = Gradient(nLiqSiO2=nLiqSiO2, nLiqAl2O3=nLiqAl2O3)
for i in range(0,g.shape[0]):
    print ("{0:10.3e}".format(g[i][0]))
print ("Lambda:")
xLambda = np.linalg.lstsq(A.transpose(),g)[0]
for i in range(0,xLambda.shape[0]):
    print ("{0:10.3e}".format(xLambda[i][0]))

### Compute the Wronskian of the Lagrangian

In [ ]:
W = Wronskian(nLiqSiO2=nLiqSiO2, nLiqAl2O3=nLiqAl2O3)
for i in range(0,W.shape[0]):
    print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e} {6:10.3e}".format( \
        W[i][0], W[i][1], W[i][2], W[i][3], W[i][4], W[i][5], W[i][6]))

### Next project the gradient and the Wronskian
The search direction ($\Delta {\bf{n}}_{_\phi }^i$) is given by
$$
{{\bf{Z}}^T}{\bf{WZ}}\Delta {\bf{n}}_\phi ^i =  - {{\bf{Z}}^T}{\bf{g}}
$$
and ${\bf{n}}_\phi ^{i + 1} = \Delta {\bf{n}}_{_\phi }^i + {\bf{n}}_\phi ^i$ gives the quadratic approximation to the minimum, which must be adjusted to maintain the non-linear equality constraints
$$
{\bf{\tilde n}}_\phi ^{i + 1} = s\Delta {\bf{n}}_\phi ^i + {\bf{n}}_\phi ^i + \Delta {\bf{n}}_\phi ^{corr}
$$

In [ ]:
ZTg = np.matmul(Z.transpose(),g)
ZTWZ = np.matmul(Z.transpose(), np.matmul(W,Z))
print("Z^T g")
for i in range(0,ZTg.shape[0]):
    print ("{0:10.3e}".format(ZTg[i][0]))
print("Z^T W Z")
for i in range(0,ZTWZ.shape[0]):
    print ("{0:10.3e} {1:10.3e}".format(ZTWZ[i][0], ZTWZ[i][1]))

### ... and solve for the correction vector

In [ ]:
x = lin.solve(ZTWZ, ZTg)
print ("Solution in the null space:")
for i in range(0,x.shape[0]):
    print ("{0:10.3e}".format(x[i][0]))
deltan = np.matmul(Z,x)
print ("Inflated Solution:")
for i in range(0,deltan.shape[0]):
    print ("{0:10.3e}".format(deltan[i][0]))

## Final solution
The correction vector is zero because the initial guess satisfies all constraints and the number of constraints is equal to the number of unknowns. 

In [ ]:
nLiq = setMoles(nSiO2=nLiqSiO2, nAl2O3=nLiqAl2O3)
print ("n SiO2    liq = {0:8.6f}".format(nLiq[0][0]+deltan[0][0]))
print ("n Al2O3   liq = {0:8.6f}".format(nLiq[1][0]+deltan[1][0]))
print ("n CaSiO3  liq = {0:8.6f}".format(nLiq[2][0]+deltan[2][0]))
print ("n Na2SiO3 liq = {0:8.6f}".format(nLiq[3][0]+deltan[3][0]))
print ("n KAlSiO4 liq = {0:8.6f}".format(nLiq[4][0]+deltan[4][0]))
print ("n Qz          = {0:8.6f}".format(nQz+deltan[5][0]))
print ("n Cr          = {0:8.6f}".format(nCr+deltan[6][0]))